In [1]:
import os
import sys
import operator
import itertools
import heapq
from datetime import datetime
from argparse import ArgumentParser
from collections import defaultdict
import pycosat
import functools
from functools import reduce
import pandas as pd
import time
import collections
from collections import Counter


LEFT = 1
RIGHT = 2
TOP = 4
BOTTOM = 8

In [2]:
DELTAS = [(LEFT, 0, -1),
          (RIGHT, 0, 1),
          (TOP, -1, 0),
          (BOTTOM, 1, 0)]

LR = LEFT | RIGHT
TB = TOP | BOTTOM
TL = TOP | LEFT
TR = TOP | RIGHT
BL = BOTTOM | LEFT
BR = BOTTOM | RIGHT

DIR_TYPES = [LR, TB, TL, TR, BL, BR]

DIR_FLIP = {
    LEFT: RIGHT,
    RIGHT: LEFT,
    TOP: BOTTOM,
    BOTTOM: TOP
    }

ANSI_LOOKUP = dict(R=101, B=104, Y=103, G=42,
                   O=43, C=106, M=105, m=41,
                   P=45, A=100, W=107, g=102,
                   T=47, b=44, c=46, p=35)

ANSI_RESET = '\033[0m'

ANSI_CELL_FORMAT = '\033[30;{}m'

DIR_LOOKUP = {
    LR: '─',
    TB: '│',
    TL: '┘',
    TR: '└',
    BL: '┐',
    BR: '┌'
    }

RESULT_STRINGS = dict(s='successful',
                      f='failed',
                      u='unsolvable')

In [3]:
def parse_puzzle(options, file_or_str, filename='input'):

    '''Convert the given string or file object into a square array of
strings. Also return a dictionary which maps input characters to color
indices.

    '''

    if not isinstance(file_or_str, str):
        file_or_str = file_or_str.read()

    puzzle = file_or_str.splitlines()

    # assume size based on length of first line
    size = len(puzzle[0])

    # make sure enough lines
    if len(puzzle) < size:
        print('{}:{} unexpected EOF'.format(filename, len(puzzle)+1))
        return None, None

    # truncate extraneous lines
    puzzle = puzzle[:size]

    # count colors and build lookup
    colors = dict()
    color_count = []

    for i, row in enumerate(puzzle):
        if len(row) != size:
            print('{}:{} row size mismatch'.format(filename, i+1))
            return None, None
        for j, char in enumerate(row):
            if char.isalnum(): # flow endpoint
                if char in colors.keys():
                    color = colors[char]
                    if color_count[color]:
                        print('{}:{}:{} too many {} already'.format(filename, i+1, j, char))
                        return None, None
                    color_count[color] = 1
                else:
                    color = len(colors)
                    colors[char] = color
                    color_count.append(0)

    # check parity
    for char, color in colors.items():
        if not color_count[color]:
            print('color {} has start but no end!'.format(char))
            return None, None

  

    puzzle, colors = repair_colors(puzzle, colors)
    return puzzle, colors


In [4]:
def make_dir_vars(puzzle, start_var):

    '''Creates the direction-type SAT variables for each cell.'''

    size = len(puzzle)
    dir_vars = dict()
    num_dir_vars = 0

    for i, j, char in explode(puzzle):

        if char.isalnum(): # flow endpoint, no dir needed
            continue

        # collect bits for neighbors (TOP BOTTOM LEFT RIGHT)
        neighbor_bits = (dir_bit for (dir_bit, ni, nj)
                         in valid_neighbors(size, i, j))

        # OR them all together
        cell_flags = reduce(operator.or_, neighbor_bits, 0)

        # create a lookup for dir type vars in this cell
        dir_vars[i, j] = dict()

        for code in DIR_TYPES:
            # only add var if cell has correct flags (i.e. if cell has
            # TOP, BOTTOM, RIGHT, don't add LR).
            if cell_flags & code == code:
                num_dir_vars += 1
                dir_vars[i, j][code] = start_var + num_dir_vars

    return dir_vars, num_dir_vars

######################################################################

def make_dir_clauses2(puzzle, colors, color_var, dir_vars):

    '''Generate clauses involving the color and direction-type SAT
variables. Each free cell must be exactly one direction, and
directions imply color matching with neighbors.

    '''

    dir_clauses = []
    num_colors = len(colors)
    size = len(puzzle)

    # for each cell
    for i, j, char in explode(puzzle):

        if char.isalnum(): # flow endpoint, no dir needed
            continue

        cell_dir_dict = dir_vars[(i, j)]
        cell_dir_vars = cell_dir_dict.values()
        
        
        ##################################adding time value##########################
        # at least one direction is set in this cell
        dir_clauses.append(("time_direction: ", time.time()))
        dir_clauses.append(cell_dir_vars)
        
        ##################################adding time value##########################
        # no two directions are set in this cell
        dir_clauses.append(("time_direction_no_two: ", time.time()))
        dir_clauses.extend(no_two(cell_dir_vars))

        # for each color
        for color in range(num_colors):

            # get color var for this cell
            color_1 = color_var(i, j, color)

            # for each neighbor
            for dir_bit, n_i, n_j in all_neighbors(i, j):

                # get color var for other cell
                color_2 = color_var(n_i, n_j, color)

                # for each direction variable in this scell
                for dir_type, dir_var in cell_dir_dict.items():

                    # if neighbor is hit by this direction type
                    if dir_type & dir_bit:
                         ##################################adding time value##########################
                        # this direction type implies the colors are equal
                        dir_clauses.append(("time_direction_imply_color: ", time.time()))
                        dir_clauses.append([-dir_var, -color_1, color_2])
                        dir_clauses.append(("time_direction_imply_color: ", time.time()))
                        dir_clauses.append([-dir_var, color_1, -color_2])
                    elif valid_pos(size, n_i, n_j):
                         ##################################adding time value##########################
                        # neighbor is not along this direction type,
                        # so this direction type implies the colors are not equal
                        dir_clauses.append(("time_direction_reduce_neigbor: ", time.time()))
                        dir_clauses.append([-dir_var, -color_1, -color_2])

    return dir_clauses


In [5]:
def make_dir_clauses(puzzle, colors, color_var, dir_vars):

    '''Generate clauses involving the color and direction-type SAT
variables. Each free cell must be exactly one direction, and
directions imply color matching with neighbors.

    '''

    dir_clauses = []
    num_colors = len(colors)
    size = len(puzzle)

    # for each cell
    for i, j, char in explode(puzzle):

        if char.isalnum(): # flow endpoint, no dir needed
            continue

        cell_dir_dict = dir_vars[(i, j)]
        cell_dir_vars = cell_dir_dict.values()
        
        
        ##################################adding time value##########################
        # at least one direction is set in this cell
        #dir_clauses.append(("time_direction: ", time.time()))
        dir_clauses.append(cell_dir_vars)
        
        ##################################adding time value##########################
        # no two directions are set in this cell
       # dir_clauses.append(("time_direction_no_two: ", time.time()))
        dir_clauses.extend(no_two(cell_dir_vars))

        # for each color
        for color in range(num_colors):

            # get color var for this cell
            color_1 = color_var(i, j, color)

            # for each neighbor
            for dir_bit, n_i, n_j in all_neighbors(i, j):

                # get color var for other cell
                color_2 = color_var(n_i, n_j, color)

                # for each direction variable in this scell
                for dir_type, dir_var in cell_dir_dict.items():

                    # if neighbor is hit by this direction type
                    if dir_type & dir_bit:
                         ##################################adding time value##########################
                        # this direction type implies the colors are equal
                        #dir_clauses.append(("time_direction_imply_color: ", time.time()))
                        dir_clauses.append([-dir_var, -color_1, color_2])
                        #dir_clauses.append(("time_direction_imply_color: ", time.time()))
                        dir_clauses.append([-dir_var, color_1, -color_2])
                    elif valid_pos(size, n_i, n_j):
                         ##################################adding time value##########################
                        # neighbor is not along this direction type,
                        # so this direction type implies the colors are not equal
                        #dir_clauses.append(("time_direction_reduce_neigbor: ", time.time()))
                        dir_clauses.append([-dir_var, -color_1, -color_2])

    return dir_clauses

In [6]:
def make_color_clauses2(puzzle, colors, color_var):

    '''Generate CNF clauses entailing the N*M color SAT variables, where N
is the number of cells and M is the number of colors. Each cell
encodes a single color in a one-hot fashion.

    '''

    clauses = []
    num_colors = len(colors)
    size = len(puzzle)

    # for each cell
    for i, j, char in explode(puzzle):

        if char.isalnum(): # flow endpoint

            endpoint_color = colors[char] #valid number

            ##########################_adding_time_variable_###################### 
            # color in this cell is this one
            clauses.append(("time_endpoint_color: ", time.time()))
            clauses.append([color_var(i, j, endpoint_color)])

            # color in this cell is not the other ones
            for other_color in range(num_colors):
                if other_color != endpoint_color:
                    clauses.append(("time_endpoint_color_negetions: ", time.time()))
                    clauses.append([-color_var(i, j, other_color)])

            # gather neighbors' variables for this color
            neighbor_vars = [color_var(ni, nj, endpoint_color) for
                             _, ni, nj in valid_neighbors(size, i, j)]

            ##########################_adding_time_variable_######################
            # one neighbor has this color
            clauses.append(("time_endpoint_color_neighbor: ", time.time()))
            clauses.append(neighbor_vars)

            ##########################_adding_time_variable_######################
            # no two neighbors have this color
            clauses.append(("time_endpoint_color_neighbor_no_two: ", time.time()))
            clauses.extend(no_two(neighbor_vars))

        else:
            ##########################_adding_time_variable_######################
            # one of the colors in this cell is set
            clauses.append(("time__color_cell: ", time.time()))
            clauses.append([color_var(i, j, color)
                            for color in range(num_colors)])

            # no two of the colors in this cell are set
            cell_color_vars = (color_var(i, j, color) for
                               color in range(num_colors))
            clauses.append(("time__color_cell_no_two: ", time.time()))
            clauses.extend(no_two(cell_color_vars))

    return clauses

######################################################################

In [7]:
def make_color_clauses(puzzle, colors, color_var):

    '''Generate CNF clauses entailing the N*M color SAT variables, where N
is the number of cells and M is the number of colors. Each cell
encodes a single color in a one-hot fashion.

    '''

    clauses = []
    num_colors = len(colors)
    size = len(puzzle)

    # for each cell
    for i, j, char in explode(puzzle):

        if char.isalnum(): # flow endpoint

            endpoint_color = colors[char] #valid number

            ##########################_adding_time_variable_###################### 
            # color in this cell is this one
            #clauses.append(("time_endpoint_color: ", time.time()))
            clauses.append([color_var(i, j, endpoint_color)])

            # color in this cell is not the other ones
            for other_color in range(num_colors):
                if other_color != endpoint_color:
                    #clauses.append(("time_endpoint_color_negetions: ", time.time()))
                    clauses.append([-color_var(i, j, other_color)])

            # gather neighbors' variables for this color
            neighbor_vars = [color_var(ni, nj, endpoint_color) for
                             _, ni, nj in valid_neighbors(size, i, j)]

            ##########################_adding_time_variable_######################
            # one neighbor has this color
           # clauses.append(("time_endpoint_color_neighbor: ", time.time()))
            clauses.append(neighbor_vars)

            ##########################_adding_time_variable_######################
            # no two neighbors have this color
            #clauses.append(("time_endpoint_color_neighbor_no_two: ", time.time()))
            clauses.extend(no_two(neighbor_vars))

        else:
            ##########################_adding_time_variable_######################
            # one of the colors in this cell is set
            #clauses.append(("time__color_cell: ", time.time()))
            clauses.append([color_var(i, j, color)
                            for color in range(num_colors)])

            # no two of the colors in this cell are set
            cell_color_vars = (color_var(i, j, color) for
                               color in range(num_colors))
            #clauses.append(("time__color_cell_no_two: ", time.time()))
            clauses.extend(no_two(cell_color_vars))

    return clauses

######################################################################

In [8]:
def explode(puzzle):
    '''Iterator helper function to allow looping over 2D arrays without
nested 'for' loops.

    '''

    for i, row in enumerate(puzzle):
        for j, char in enumerate(row):
            yield i, j, char

In [9]:
def no_two(satvars):
    '''Given a collection of SAT variables, generates clauses specifying
that no two of them can be true at the same time.

    '''

    return ((-a, -b) for (a, b) in all_pairs(satvars))

In [10]:
def all_pairs(collection):
    '''Return all combinations of two items from a collection, useful for
making a large number of SAT variables mutually exclusive.

    '''

    return itertools.combinations(collection, 2)


In [11]:
def valid_pos(size, i, j):
    '''Check whether a position on a square grid is valid.'''
    return i >= 0 and i < size and j >= 0 and j < size

######################################################################

def all_neighbors(i, j):
    '''Return all neighbors of a grid square at row i, column j.'''
    return ((dir_bit, i+delta_i, j+delta_j)
            for (dir_bit, delta_i, delta_j) in DELTAS)

######################################################################

def valid_neighbors(size, i, j):
    '''Return all actual on-grid neighbors of a grid square at row i,
column j.'''

    return ((dir_bit, ni, nj) for (dir_bit, ni, nj)
            in all_neighbors(i, j)
            if valid_pos(size, ni, nj))

In [12]:
def repair_colors(puzzle, colors):

    '''If the puzzle file used "color labels" (A,B,C...), instead
    of color mnemonics (R,G,B...), convert the labels to mnemonics.
    Note: If a puzzle is in mnemonic format, it will contain the
    letter 'R' since Red is always the first color. Absence of an 'R'
    is the test for a color-labeled puzzle.
    '''
    if 'R' in colors.keys():
        return puzzle, colors

    color_lookup = 'RBYGOCMmPAWgTbcp'
    new_puzzle = []

    try:

        for row in puzzle:

            new_row = []

            for char in row:
                if char.isalnum():
                    char = color_lookup[ord(char)-ord('A')]
                new_row.append(char)

            new_puzzle.append(''.join(new_row))

        new_colors = dict((color_lookup[ord(char)-ord('A')], index)
                          for (char, index) in colors.items())

    except IndexError:
        return puzzle, colors

    return new_puzzle, new_colors

######################################################################

In [13]:
def decode_solution(puzzle, colors, color_var, dir_vars, sol):

    '''Takes the solution set from SAT and decodes it by undoing the
one-hot encoding in each cell for color and direction-type. Returns a
2D array of (color, direction-type) pairs.

    '''

    sol = set(sol)
    num_colors = len(colors)

    decoded = []

    for i, row in enumerate(puzzle):

        decoded_row = []

        for j, char in enumerate(row):

            # find which color variable for this cell is in the
            # solution set
            cell_color = -1

            for color in range(num_colors):
                if color_var(i, j, color) in sol:
                    assert cell_color == -1
                    cell_color = color

            assert cell_color != -1

            cell_dir_type = -1

            if not char.isalnum(): # not a flow endpoint

                # find which dir type variable for this cell is in the
                # solution set
                for dir_type, dir_var in dir_vars[i, j].items():
                    if dir_var in sol:
                        assert cell_dir_type == -1
                        cell_dir_type = dir_type

                assert cell_dir_type != -1

            decoded_row.append((cell_color, cell_dir_type))

        decoded.append(decoded_row)

    return decoded


In [14]:
def reduce_to_sat2(options, puzzle, colors):

    '''Reduces the given puzzle to a SAT problem specified in CNF. Returns
a list of clauses where each clause is a list of single SAT variables,
possibly negated.

    '''

    size = len(puzzle)
    num_colors = len(colors)

    num_cells = size**2
    num_color_vars = num_colors * num_cells

    def color_var(i, j, color):
        '''Return the index of the SAT variable for the given color in row i,
 column j.

        '''
        return (i*size + j)*num_colors + color + 1

    start = datetime.now()

    color_clauses = make_color_clauses2(puzzle,
                                       colors,
                                       color_var)

    dir_vars, num_dir_vars = make_dir_vars(puzzle, num_color_vars)

    dir_clauses = make_dir_clauses2(puzzle, colors,
                                   color_var, dir_vars)

    num_vars = num_color_vars + num_dir_vars
    clauses = color_clauses + dir_clauses

    reduce_time = (datetime.now() - start).total_seconds()

    

    return color_var, dir_vars, num_vars, clauses, reduce_time

######################################################################

In [15]:
def reduce_to_sat(options, puzzle, colors):

    '''Reduces the given puzzle to a SAT problem specified in CNF. Returns
a list of clauses where each clause is a list of single SAT variables,
possibly negated.

    '''

    size = len(puzzle)
    num_colors = len(colors)

    num_cells = size**2
    num_color_vars = num_colors * num_cells

    def color_var(i, j, color):
        '''Return the index of the SAT variable for the given color in row i,
 column j.

        '''
        return (i*size + j)*num_colors + color + 1

    start = datetime.now()

    color_clauses = make_color_clauses(puzzle,
                                       colors,
                                       color_var)

    dir_vars, num_dir_vars = make_dir_vars(puzzle, num_color_vars)

    dir_clauses = make_dir_clauses(puzzle, colors,
                                   color_var, dir_vars)

    num_vars = num_color_vars + num_dir_vars
    clauses = color_clauses + dir_clauses

    reduce_time = (datetime.now() - start).total_seconds()

    

    return color_var, dir_vars, num_vars, clauses, reduce_time

######################################################################

In [16]:
def solve_sat(options, puzzle, colors, color_var, dir_vars, clauses):

    '''Solve the SAT now that it has been reduced to a list of clauses in
CNF.  This is an iterative process: first we try to solve a SAT, then
we detect cycles. If cycles are found, they are prevented from
recurring, and the next iteration begins. Returns the SAT solution
set, the decoded puzzle solution, and the number of cycle repairs
needed.

    '''

    start = datetime.now()

    decoded = None
    all_decoded = []
    repairs = 0

    while True:

        sol = pycosat.solve(clauses) # pylint: disable=E1101

        if not isinstance(sol, list):
            decoded = None
            all_decoded.append(decoded)
            break

        decoded = decode_solution(puzzle, colors, color_var, dir_vars, sol)
        all_decoded.append(decoded)

#         extra_clauses = detect_cycles(decoded, dir_vars)

#         if not extra_clauses:
#             break

#         clauses += extra_clauses
#         repairs += 1

#     solve_time = (datetime.now() - start).total_seconds()

#     if not options.quiet:
#         if options.display_cycles:
#             for cycle_decoded in all_decoded[:-1]:
#                 print('intermediate solution with cycles:')
#                 show_solution(options, colors, cycle_decoded)


#         if decoded is None:
#             print('solver returned {} after {:,} cycle repairs and {:.3f} seconds'.format(str(sol), repairs, solve_time))
#         else:
#             print('obtained solution after {:,} cycle repairs and {:.3f} seconds:'.format(repairs, solve_time))
#             show_solution(options, colors, decoded)

    return sol, decoded, repairs, solve_time

In [17]:
def print_summary(filename, stats):

    '''Print out stats for all solutions.'''
    
    max_width = max(len(f) for f in filename)
    

    solution_types = stats.keys()

    all_stats = defaultdict(float)
   
    for result_char in solution_types:
        for k in stats[result_char]:
            all_stats[k] += stats[result_char][k]
            
    print(stats)

#     if all_stats['count'] > 1:
#         print('t')

    print('\n'+('*'*70)+'\n')

    for result_char in solution_types:

        print('{:d} {:s} searches took:\n'\
            '  {:,.3f} sec. to reduce '\
            '(with {:,d} variables and {:,d} clauses)\n'\
            '  {:,.3f} sec. to solve (with {:d} repairs)\n'\
            '  {:,.3f} sec. total\n'.format(
                stats[result_char]['count'], result_char,
                stats[result_char]['reduce_time'],
                stats[result_char]['num_vars'],
                stats[result_char]['num_clauses'],
                stats[result_char]['solve_time'],
                stats[result_char]['repairs'],
                stats[result_char]['total_time']))

        

        print('overall, {:d} searches took:\n'\
            '  {:,.3f} sec. to reduce '\
            '(with {:,d} variables and {:,d} clauses)\n'\
            '  {:,.3f} sec. to solve (with {:d} repairs)\n'\
            '  {:,.3f} sec. total\n'.format(
                int(all_stats['count']),
                all_stats['reduce_time'],
                int(all_stats['num_vars']),
                int(all_stats['num_clauses']),
                all_stats['solve_time'],
                int(all_stats['repairs']),
                all_stats['total_time']))

        for result_char in solution_types:

            print('{:s}{:3d} total {:s} {:9,d} {:9,d} {:12,.3f} '\
                '{:3d} {:12,.3f} {:12,.3f}'.format(
                    ' '*(max_width-9), stats[result_char]['count'],
                    result_char,
                    stats[result_char]['num_vars'],
                    stats[result_char]['num_clauses'],
                    stats[result_char]['reduce_time'],
                    stats[result_char]['repairs'],
                    stats[result_char]['solve_time'],
                    stats[result_char]['total_time']))

#         if len(solution_types) > 1:

        print('{:s}{:3d} overall {:9,d} {:9,d} {:12,.3f} '\
            '{:3d} {:12,.3f} {:12,.3f}'.format(
                ' '*(max_width-9), int(all_stats['count']),
                int(all_stats['num_vars']), int(all_stats['num_clauses']),
                all_stats['reduce_time'], int(all_stats['repairs']),
                all_stats['solve_time'], all_stats['total_time']))

######################################################################

In [18]:
    def check_circle(clauses,all_times):        
        while(True):
            for i, item in enumerate(clauses):
                if (type(item)==tuple and (item[1])>(1589745930)):
                    for j, val in enumerate(clauses):
                         if (type(val)==tuple and val[1]>(1589745930)):
                            all_times[(item[0],item[1])]=clauses[i+1:j-1]

In [19]:
def is_circle(directions_and_color):
    for v in directions_and_color.values():
        for v2 in directions_and_color.values():
            if v!= v2:
                list_values_v=[t for t in v if  type(t)==list ]
                list_values_v2=[t2 for t2 in v2 if  type(t2)==list ]
                for item in list_values_v:
                    for item2 in list_values_v2:
                        numN=(abs(item[0]))
                        num=[int(i) for i in str(numN)]
                        num2N=(abs(item2[0]))
                        num2=[int(i) for i in str(num2N)]
                        if (((num[0]==1+num2[0]) or(num[1]==1+num2[1])or (num[2]==1+num2[2]) )and item[1]==item2[1] and item[2]==item2[2]):
                            print('circle')
                            return True
    return False

In [20]:
def pyflow_solver_main():
    
    filename=r"C:\Users\vicli\Desktop\תמר לימודים\תואר שני\thesis\documation\sudoku\sudoku-sat-solver\flow game using sat\flow_solver_with_sat\puzzles\extreme_8x8_01.txt"
    
    options=''
    
    stats = {}
    with open(filename, 'r') as infile:
                puzzle, colors = parse_puzzle(options, infile, filename)

    start = time.time()
    color_var, dir_vars, num_vars, clauses2, reduce_time = reduce_to_sat2(options, puzzle, colors)
    color_var, dir_vars, num_vars, clauses, reduce_time = reduce_to_sat(options, puzzle, colors)
    all_times={}
    
#     for clause in clauses2:
# #         print('t')
#          if (type(clause)==tuple):
#             print('\nclause:\n', clause)
# #             for clause2 in ((clauses2)):
# #                 if type(clause2)==tuple and 'time' in clause2:
# #                     print(clause2)
# #                    # all_time[clause[1]]= 
    
#     def check_circle(clauses2):  
#         print('t')
#         while(True):
#             for i, item in enumerate(clauses):
# #                 print(item)
#                 if (type(item)==tuple and (item[1])>([all_times.keys()][0])):
#                     for j, val in enumerate(clauses):
#                          if (type(val)==tuple and val[1]>([all_times.keys()][0])):
#                             all_times[(item[0],item[1])]=clauses[i+1:j-1]

#             directions_and_color={}
#             #taking only clauses of "time_direction_direction_imply_color":
#             for key in all_times.keys():
#                 print(key[0])
#                 if key[0]=='time_direction_direction_imply_color: ':
#                     directions_and_color[key[1]]=all_times[key]

#                 circle=is_circle(directions_and_color)
#                 if not circle:
#                     break
#                 return clauses
        
#     clauses=check_circle(clauses2)
    print(clauses)

    
#         #########befor here. to think how i can check cycles here
    sol, _, repairs, solve_time = solve_sat(options, puzzle, colors,color_var, dir_vars, clauses)


    total_time = reduce_time + solve_time

    if isinstance(sol, list):
        result_char = 's'
    elif str(sol) == 'UNSAT':
        result_char = 'u'
    else:
        result_char = 'f'

    cur_stats = dict(repairs=repairs,
                     reduce_time=reduce_time,
                     solve_time=solve_time,
                     total_time=total_time,
                     num_vars=num_vars,
                     num_clauses=len(clauses),
                     count=1)

    if not result_char in stats.keys():
        stats[result_char] = cur_stats
    else:
        for key in cur_stats.keys():
            stats[result_char][key] += cur_stats[key]

    if not options.quiet:
        print('finished in total of {:.3f} seconds'.format(total_time))
    else:

        print('{:>{}s} {} {:9,d} {:9,d} {:12,.3f} '\
            '{:3d} {:12,.3f} {:12,.3f}'.format(
                filename, max_width, result_char,
                num_vars, len(clauses), reduce_time,
                repairs, solve_time, total_time))

    print(clauses, stats)

    print_summary(filename, stats)



In [ ]:
if __name__ == '__main__':
    
    pyflow_solver_main()
 
        
            
    

[[1, 2, 3, 4, 5], (-1, -2), (-1, -3), (-1, -4), (-1, -5), (-2, -3), (-2, -4), (-2, -5), (-3, -4), (-3, -5), (-4, -5), [6, 7, 8, 9, 10], (-6, -7), (-6, -8), (-6, -9), (-6, -10), (-7, -8), (-7, -9), (-7, -10), (-8, -9), (-8, -10), (-9, -10), [11, 12, 13, 14, 15], (-11, -12), (-11, -13), (-11, -14), (-11, -15), (-12, -13), (-12, -14), (-12, -15), (-13, -14), (-13, -15), (-14, -15), [16, 17, 18, 19, 20], (-16, -17), (-16, -18), (-16, -19), (-16, -20), (-17, -18), (-17, -19), (-17, -20), (-18, -19), (-18, -20), (-19, -20), [21, 22, 23, 24, 25], (-21, -22), (-21, -23), (-21, -24), (-21, -25), (-22, -23), (-22, -24), (-22, -25), (-23, -24), (-23, -25), (-24, -25), [26, 27, 28, 29, 30], (-26, -27), (-26, -28), (-26, -29), (-26, -30), (-27, -28), (-27, -29), (-27, -30), (-28, -29), (-28, -30), (-29, -30), [31, 32, 33, 34, 35], (-31, -32), (-31, -33), (-31, -34), (-31, -35), (-32, -33), (-32, -34), (-32, -35), (-33, -34), (-33, -35), (-34, -35), [36, 37, 38, 39, 40], (-36, -37), (-36, -38), (-36

In [78]:
#(i*size+j)*numcolors +color +1


In [72]:
all_times

NameError: name 'all_times' is not defined

In [ ]:
# create new clouses that combined no - | - | in the same color one after anothe

In [45]:

directions_and_color={}
#taking only clauses of "time_direction_direction_imply_color":
for key in all_times.keys():
    print(key[0])
    if key[0]=='time_direction_direction_imply_color: ':
        directions_and_color[key[1]]=all_times[key]

NameError: name 'all_times' is not defined

In [68]:
directions_and_color
    

{1589834045.6364648: [[-321, 1, -41],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -2, 7],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, 2, -7],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -2, 42],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, 2, -42],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -3, 8],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, 3, -8],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -3, 43],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, 3, -43],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -4, 9],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, 4, -9],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -4, 44],
  ('time_direction_direction_imply_color: ',

In [69]:
directions_and_color.keys()

dict_keys([1589834045.6364648, 1589834045.6374643, 1589834045.6384628, 1589834045.6384883, 1589834045.63949, 1589834045.6403875, 1589834045.6413882, 1589834045.6420665, 1589834045.6430666, 1589834045.6471267, 1589834045.6592376, 1589834045.6609497])

In [78]:
# #for k,v in directions_and_color.items():
# good = [t for t in directions_and_color.values() if not 'time' in (t) ]

In [89]:
(list(directions_and_color.values()))

[[[-321, 1, -41],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -2, 7],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, 2, -7],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -2, 42],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, 2, -42],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -3, 8],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, 3, -8],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -3, 43],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, 3, -43],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -4, 9],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, 4, -9],
  ('time_direction_direction_imply_color: ', 1589834045.6374643),
  [-321, -4, 44],
  ('time_direction_direction_imply_color: ', 1589834045.6374643)